In [1]:
from datasets import Dataset
import torch
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import (
    Compose,
    Normalize,
    ToTensor
)
from tqdm import tqdm

c:\app\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import random_split
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
os.chdir('..')

In [4]:
BATCH_SIZE=64
num_epochs=100
lr=1e-4
class_size=5

In [5]:
def dataset_generator():
    """
    Generator for datasets 

    Parametrs:
    path (str) : path to folder
    """
    labels: list = ["Disgust", "Fear", "Sad", "Netural", "Happy"]
    for file in os.listdir("spectrograms"):
        tokens: list = file.split("_")
        label: int = int(tokens[1])
        spectrogram: np.array = np.load(os.path.join('spectrograms', file))
        spectrogram = 10*np.log10(spectrogram)
        spectrogram = (spectrogram - np.min(spectrogram)) / (np.max(spectrogram) - np.min(spectrogram)) * 255
        spectrogram = spectrogram.astype(np.uint8)
        spectrogram = Image.fromarray(spectrogram, mode="L")
        yield {
            "img": spectrogram,
            "label": labels[label] 
        }

dataset = Dataset.from_generator(dataset_generator)
dataset = dataset.class_encode_column("label")

In [6]:
id2label = {id:label for id, label in enumerate(dataset.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}
id2label

{0: 'Disgust', 1: 'Fear', 2: 'Happy', 3: 'Netural', 4: 'Sad'}

In [7]:
splits_train_test = dataset.train_test_split(test_size = 0.3)
train_ds = splits_train_test['train']
val_test_ds = splits_train_test['test']
test_ds = val_test_ds.train_test_split(test_size=0.5)['train']
val_ds = val_test_ds.train_test_split(test_size=0.5)['test']

In [8]:
_ds_transform = Compose([
    ToTensor()
])

def ds_transform(examples):
    examples['img'] = [_ds_transform(image.convert("L")) for image in examples['img']]
    return examples

train_ds.set_transform(ds_transform)
test_ds.set_transform(ds_transform)
val_ds.set_transform(ds_transform)

In [9]:
class VGG16_NET(nn.Module):
    def __init__(self):
        super(VGG16_NET, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.conv8 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv9 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.conv11 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv13 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc14 = nn.Linear(2048, 1024)
        self.fc15 = nn.Linear(1024, 512)
        self.fc16 = nn.Linear(512, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.maxpool(x)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.maxpool(x)
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = self.maxpool(x)
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc14(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.fc15(x))
        x = F.dropout(x, 0.5)
        x = F.softmax(self.fc16(x))
        return x

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
model = VGG16_NET() 
model = model.to(device=device) 
load_model = True
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= lr) 

In [11]:
train_dataloader = DataLoader(dataset=train_ds,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

val_dataloader = DataLoader(dataset=val_ds,
                            batch_size=BATCH_SIZE,
                            shuffle=True)

test_dataloader = DataLoader(dataset=test_ds,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

In [12]:
for epoch in range(num_epochs): #I decided to train the model for 50 epochs
    loss_var = 0
    
    for idx, image_label in enumerate(tqdm(train_dataloader, position = 0, leave = True)):
        img = image_label['img']
        labels = image_label['label']
        images = img.to(device=device)
        labels = labels.to(device=device)
        ## Forward Pass
        optimizer.zero_grad()
        scores = model(images)
        
        loss = criterion(scores,labels)
        loss.backward()
        optimizer.step()
        loss_var += loss.item()
        if idx%64==0:
            print(f'Epoch [{epoch+1}/{num_epochs}] || Step [{idx+1}/{len(train_dataloader)}] || Loss:{loss_var/len(train_dataloader)}')
    print(f"Loss at epoch {epoch+1} || {loss_var/len(train_dataloader)}")

    with torch.no_grad():
        correct = 0
        samples = 0
        for idx, image_label in enumerate(tqdm(val_dataloader, position = 0, leave = True)):
            img = image_label['img']
            labels = image_label['label']
            images = img.to(device=device)
            labels = labels.to(device=device)
            outputs = model(images)
            _, preds = outputs.max(1)
            correct += (preds == labels).sum()
            samples += preds.size(0)
        print(f"accuracy {float(correct) / float(samples) * 100:.2f} percentage || Correct {correct} out of {samples} samples")

  0%|          | 0/441 [00:00<?, ?it/s]C:\Users\z0224841\AppData\Local\Temp\ipykernel_1776\178145702.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc16(x))
  0%|          | 1/441 [00:01<08:10,  1.11s/it]

Epoch [1/100] || Step [1/441] || Loss:0.0036497583735286515


 15%|█▍        | 65/441 [01:07<06:35,  1.05s/it]

Epoch [1/100] || Step [65/441] || Loss:0.2368810214693584


 29%|██▉       | 129/441 [02:12<04:59,  1.04it/s]

...

Epoch [100/100] || Step [385/441] || Loss:1.3956488443880666


100%|██████████| 441/441 [06:35<00:00,  1.12it/s]


Loss at epoch 100 || 1.5978282568406086


100%|██████████| 95/95 [00:27<00:00,  3.45it/s]

accuracy 24.98 percentage || Correct 1508 out of 6036 samples


In [13]:
torch.save(model.state_dict(), "vgg16_model.pt") #SAVES THE TRAINED MODEL
model = VGG16_NET()
model.load_state_dict(torch.load("vgg16_model.pt")) #loads the trained model
model.eval()

VGG16_NET(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv9): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv10): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv11): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv12): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv13): Conv2d(512, 512, kernel_size=(3,